In [1]:
from sklearn import *
import lightgbm as lgbm
import xgboost as xgb
import catboost as cat
from sklearn.model_selection import train_test_split
from sqlalchemy import column
import pandas as pd
import numpy as np
#!pip install optuna 
import optuna

from sklearn.compose import make_column_transformer
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.impute import KNNImputer
from sklearn.impute import SimpleImputer
from sklearn.linear_model import BayesianRidge

from lightgbm import LGBMRegressor
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_squared_log_error
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

In [2]:
def convert_to_category(X):
    for col in X.columns:
        if ( (X[col].dtype != 'int64') and (X[col].dtype != 'float64') and (X[col].dtype != 'bool')):
            X[col] = X[col].astype('category')
            
    return X

def remove_outliers(df, column_names, n):
    for column_name in column_names:
        mean = df[column_name].mean()
        std = df[column_name].std()
        df = df[(df[column_name] > mean - n*std) & (df[column_name] < mean + n*std)]
    return df
    
def log_func(y, shift_var):
    return np.log(y+shift_var)

def exp_func(y, shift_var):
    return np.exp(y)-shift_var

In [3]:
import numpy as np 

def rmsle(y_true, y_pred):
    """
    Computes the Root Mean Squared Logarithmic Error 
    
    Args:
        y_true (np.array): n-dimensional vector of ground-truth values 
        y_pred (np.array): n-dimensional vecotr of predicted values 
    
    Returns:
        A scalar float with the rmsle value 
    
    Note: You can alternatively use sklearn and just do: 
        `sklearn.metrics.mean_squared_log_error(y_true, y_pred) ** 0.5`
    """
    assert (y_true >= 0).all(), 'Received negative y_true values'
    assert (y_pred >= 0).all(), 'Received negative y_pred values'
    assert y_true.shape == y_pred.shape, 'y_true and y_pred have different shapes'
    y_true_log1p = np.log1p(y_true)  # log(1 + y_true)
    y_pred_log1p = np.log1p(y_pred)  # log(1 + y_pred)
    return np.sqrt(np.mean(np.square(y_pred_log1p - y_true_log1p)))

In [11]:
stores_train = pd.read_csv('data/stores_train_with_extra_features.csv')
stores_test = pd.read_csv('data/stores_test_with_extra_features.csv')

plaace_hierarchy = pd.read_csv('data/plaace_hierarchy.csv')
grunnkrets = pd.read_csv('data/grunnkrets_norway_stripped.csv')
grunnkrets_ages = pd.read_csv('data/grunnkrets_age_distribution.csv')
grunnkrets_household_types = pd.read_csv('data/grunnkrets_households_num_persons.csv')
grunnkrets_household_income = pd.read_csv('data/grunnkrets_income_households.csv')

stores_train_copy= stores_train.copy()

plaace_hierarchy_copy = plaace_hierarchy.copy()
plaace_hierarchy_copy.drop(columns='sales_channel_name', inplace=True)
plaace_hierarchy_copy['lv1'] = plaace_hierarchy_copy['lv1'].astype('category')
plaace_hierarchy_copy['lv2'] = plaace_hierarchy_copy['lv2'].astype('category')


grunnkrets_copy = grunnkrets.copy()
grunnkrets_copy.rename(columns={'year': 'year_1'}, inplace=True)

grunnkrets_ages_copy = grunnkrets_ages.copy()
grunnkrets_ages_copy.rename(columns={'year': 'year_2'}, inplace=True)
grunnkrets_ages_copy['grunnkrets_population'] = grunnkrets_ages_copy.iloc[:, 2:].sum(axis=1)

grunnkrets_household_types_copy = grunnkrets_household_types.copy()
grunnkrets_household_types_copy.rename(columns={'year': 'year_3'}, inplace=True)
grunnkrets_household_types_copy[grunnkrets_household_types_copy.columns[2:]] = grunnkrets_household_types_copy[grunnkrets_household_types_copy.columns[2:]].astype('int64')

grunnkrets_household_income_copy = grunnkrets_household_income.copy()
grunnkrets_household_income_copy.rename(columns={'year': 'year_4', 'singles': 'singles_income','couple_without_children':'couple_without_children_income'}, inplace=True)
grunnkrets_household_income_copy.rename(columns={'singles': 'singles_income', }, inplace=True)

#set the values that are 0 to the lowest value in the column
# for column in grunnkrets_household_income_copy.columns[2:]:
#     grunnkrets_household_income_copy[column] = grunnkrets_household_income_copy[column].apply(lambda x: grunnkrets_household_income_copy[column].min() if x == 0 else x)
    

df = stores_train_copy

df = pd.merge(df, plaace_hierarchy_copy, on='plaace_hierarchy_id', how='left')
df = pd.merge(df, grunnkrets_copy, on='grunnkrets_id', how='left')
df = pd.merge(df, grunnkrets_ages_copy, on='grunnkrets_id', how='left')
df = pd.merge(df, grunnkrets_household_types_copy, on='grunnkrets_id', how='left')
df = pd.merge(df, grunnkrets_household_income_copy, on='grunnkrets_id', how='left')



df['grunnkrets_population_density'] = df['grunnkrets_population'] / df['area_km2']



df.drop_duplicates(subset=['store_id'], keep='first', inplace=True)
df.reset_index(drop=True, inplace=True)



columns_to_drop = ['revenue',
                  'store_id',
                  'plaace_hierarchy_id',
                  'grunnkrets_id',
                  'year',
                   'address',
                  'store_name',
                  
                  'year_1',
                  'geometry',
                  
                  #'area_km2',
               
                  'grunnkrets_name',
                  'district_name',
             
                  'municipality_name',
                  
                  
                  'year_2',
                  'year_3',
                  'year_4',
                 
                
                  'sales_channel_name',
                   #'mall_name',
                  #'chain_name',
                  
                  
                  
                     'lv1',
                  'lv2',
                  'lv3',
                  'lv4',
                  
                  # 'lv1_desc',
                  # 'lv2_desc',
                  # 'lv3_desc',
                  # 'lv4_desc'
                
#                 'closest_busstop',
                  
#     'num_closest_busstops_250m',
#    'num_closest_busstops_500m',

#        'num_closest_busstops_1000m', 'num_closest_busstops_2500m',
#        'num_closest_busstops_5000m', 
#        'num_closest_busstops',
#        'num_closest_busstops_10000m', 'num_closest_busstops_15000m',
       
#        'closest_store_lv1', 'closest_store_lv2',
#        'closest_store_lv3', 'closest_store_lv4', 
       
#        'num_closest_stores_lv1_250m',   
#    'num_closest_stores_lv1_500m', 
   
#    'num_closest_stores_lv1_1000m',
    
#     'num_closest_stores_lv1_2500m',
#        'num_closest_stores_lv1_5000m', 
#        'num_closest_stores_lv1',
#        'num_closest_stores_lv1_10000m', 
# 'num_closest_stores_lv1_15000m',
       
#        'num_closest_stores_lv2_250m', 
   
   
#    'num_closest_stores_lv2_500m',
   
#     'num_closest_stores_lv2_1000m', 
    
#     'num_closest_stores_lv2_2500m',
#        'num_closest_stores_lv2_5000m',
#        'num_closest_stores_lv2',
#        'num_closest_stores_lv2_10000m', 'num_closest_stores_lv2_15000m',
       
#         'num_closest_stores_lv3_250m', 
   
   
#    'num_closest_stores_lv3_500m',
   
#        'num_closest_stores_lv3_1000m', 'num_closest_stores_lv3_2500m',
#        'num_closest_stores_lv3_5000m', 
#        'num_closest_stores_lv3',
#        'num_closest_stores_lv3_10000m', 'num_closest_stores_lv3_15000m',
       
#         'num_closest_stores_lv4_250m', 
   
#    'num_closest_stores_lv4_500m',
   
#        'num_closest_stores_lv4_1000m', 'num_closest_stores_lv4_2500m',
#        'num_closest_stores_lv4_5000m', 
#        'num_closest_stores_lv4',
#        'num_closest_stores_lv4_10000m', 'num_closest_stores_lv4_15000m',
#                   'distance_to_oslo',
                  'distance_to_bergen',
                  'distance_to_trondheim',
                  'distance_to_stavanger',
                  'distance_to_drammen',
                  # 'grunnkrets_population',
                  # 'grunnkrets_population_density',
                  
                  # 'num_closest_busstops_100m',
                  # 'num_closest_busstops_750m',
                  # 'num_closest_stores_lv1_100m',
                  # 'num_closest_stores_lv1_750m', 
                  # 'num_closest_stores_lv2_100m',
                  # 'num_closest_stores_lv2_750m',
                  # 'num_closest_stores_lv3_100m',
                  # 'num_closest_stores_lv3_750m',
              
                  # 'num_closest_stores_lv4_750m',
                  
                  ]


# fill in missing values of float columns with mean


X = df.drop(columns=columns_to_drop)
X = convert_to_category(X)

# for column in numerical_columns:
#    X[column] = np.log(X[column]+0.001)


y = df.revenue
y = np.log1p(y)




categorical_features = X.select_dtypes(include=['category']).columns
numerical_columns = X.select_dtypes(include=['int64', 'float64']).columns


# One hot encoding
        
full_pipeline = ColumnTransformer([('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)], remainder='passthrough')
one_hot_encoder = full_pipeline.fit(X)
X_encoded_one_hot = one_hot_encoder.transform(X)


In [24]:
random_state = 12

X_selected = X_encoded_one_hot.copy()
y_selected = y.copy()


def objective(trial):
    

    param = {
        'random_state': random_state,
        'max_depth': trial.suggest_int('max_depth', 1, 10),
        'learning_rate': trial.suggest_loguniform('learning_rate', 0.01, 1.0),
        'n_estimators': trial.suggest_int('n_estimators', 10, 1000),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),
        'gamma': trial.suggest_loguniform('gamma', 1e-8, 1.0),
        'subsample': trial.suggest_loguniform('subsample', 0.01, 1.0),
        'colsample_bytree': trial.suggest_loguniform('colsample_bytree', 0.01, 1.0),
        'reg_alpha': trial.suggest_loguniform('reg_alpha', 1e-8, 50.0),
        'reg_lambda': trial.suggest_loguniform('reg_lambda', 1e-8, 50.0),
        'eval_metric': 'rmse',

    }
    
    kf = KFold(n_splits=5, shuffle=True, random_state=42)

    kf_scores = np.empty(5)
    
    model = xgb.XGBRegressor(**param)
    
   
    for idx, (train_index, test_index) in enumerate(kf.split(X_selected, y_selected)):
        X_train, X_test = X_selected[train_index], X_selected[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]
  
        model.fit(X_train, y_train)
    
        preds = model.predict(X_test)
        
        y_test_exp = np.expm1(y_test)
        preds_exp = np.expm1(preds)
        
        preds_exp = np.where(preds_exp < 0, 0, preds_exp)
        
        kf_scores[idx] = rmsle(y_test_exp, preds_exp)

    return np.mean(kf_scores)

In [25]:
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=500)
print('Number of finished trials:', len(study.trials))
print('Best trial:', study.best_trial.params)
print('Best value:', study.best_value)


[I 2022-11-10 08:25:28,325] A new study created in memory with name: no-name-278b2caa-76de-411b-8690-f1e1cbc1e2f9
[I 2022-11-10 08:25:50,172] Trial 0 finished with value: 0.9587271024235754 and parameters: {'max_depth': 4, 'learning_rate': 0.08014804375756202, 'n_estimators': 736, 'min_child_weight': 6, 'gamma': 0.2219096850518213, 'subsample': 0.020586660763643023, 'colsample_bytree': 0.7213563662381751, 'reg_alpha': 0.013080884925409547, 'reg_lambda': 0.028461700838635747}. Best is trial 0 with value: 0.9587271024235754.
[I 2022-11-10 08:25:53,706] Trial 1 finished with value: 0.7628470020527003 and parameters: {'max_depth': 4, 'learning_rate': 0.1656419978378039, 'n_estimators': 145, 'min_child_weight': 8, 'gamma': 1.9714754355382608e-08, 'subsample': 0.2594513000245596, 'colsample_bytree': 0.16082004082813994, 'reg_alpha': 4.451712342539565e-06, 'reg_lambda': 0.39793991786757993}. Best is trial 1 with value: 0.7628470020527003.
[I 2022-11-10 08:26:05,955] Trial 2 finished with valu

Number of finished trials: 500
Best trial: {'max_depth': 3, 'learning_rate': 0.07967950419492734, 'n_estimators': 747, 'min_child_weight': 5, 'gamma': 1.4083866698278934e-07, 'subsample': 0.9999883950508703, 'colsample_bytree': 0.2348798703469321, 'reg_alpha': 9.618505724839914e-08, 'reg_lambda': 1.5088122698302632e-07}
Best value: 0.725210174385903


In [26]:
params = study.best_trial.params
val = study.best_trial.value

params

{'max_depth': 3,
 'learning_rate': 0.07967950419492734,
 'n_estimators': 747,
 'min_child_weight': 5,
 'gamma': 1.4083866698278934e-07,
 'subsample': 0.9999883950508703,
 'colsample_bytree': 0.2348798703469321,
 'reg_alpha': 9.618505724839914e-08,
 'reg_lambda': 1.5088122698302632e-07}

In [ ]:
import json
params = study.best_trial.params
val = study.best_trial.value

dict = {'val': val, 'params': params}

# save the params and value

with open('xgb_params.json', 'w') as fp:
    json.dump(dict, fp)